In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
from itertools import product

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from dataloader.builder import build_dataset
from model.mlp import MLP
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from experiment_setup import build_estimator, get_model
from experiments.utils.data import scale, split_ood, multiple_kfold
import torch

from analysis.autoencoder import AutoEncoder 

plt.rcParams['figure.facecolor'] = 'white'
torch.cuda.set_device(1)


In [ ]:
config = {
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    'k_folds': 10,
    'verbose': False,
    'layers': [8, 256, 256, 128, 1],
    'epochs': 10_000,
    'validation_step': 50,
    'acc_percentile': 0.1,
    'patience': 3,
    'dropout_rate': 0.2,
    'dropout_uq': 0.5,
    'batch_size': 256,
    'dataset': 'kin8nm',
    'ood_percentile': 90,
    'with_ensembles': True,
    'optimizer': {'type': 'Adadelta', 'weight_decay':1e-3}
}

In [ ]:

# Load dataset
dataset = build_dataset(config['dataset'])

x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')


In [ ]:
x_train, x_val, x_scaler = scale(x_train, x_val)
y_train, y_val, y_scaler = scale(y_train, y_val)

In [ ]:
autoencoder = AutoEncoder(8, 128, 2, lr=1e-2)
autoencoder.train()
for e in range(500):
    loss = autoencoder.fit(x_train)
    if (e+1) % 5 == 0:
        print(e+1, loss)


In [ ]:
autoencoder.eval()
def encode(x):
    samples = torch.DoubleTensor(x).to('cuda')
    encoded = autoencoder.encode(samples)
    return encoded.cpu().detach().numpy()

def decode(x):
    samples = torch.DoubleTensor(x).to('cuda')
    encoded = autoencoder.decode(samples)
    return encoded.cpu().detach().numpy()


In [ ]:
encoded_train = encode(x_train)
plt.figure(figsize=(12, 10))
sns.scatterplot(
    x=encoded_train[:, 0], y=encoded_train[:, 1], hue=y_train.squeeze(),
    palette="Accent")


In [ ]:
# Train or load model
model = MLP(config['layers'], optimizer=config['optimizer'])
model_path = f"experiments/data/model_{config['dataset']}.ckpt"
model = get_model(model, model_path, (x_train, y_train), (x_val, y_val))


In [ ]:
ngridx = 150
ngridy = 150
x = encoded_train[:, 0]
y = encoded_train[:, 1]

x1, x2 = 3*min(x), 3*max(x)
y1, y2 = 3*min(y), 3*max(y)
xi = np.linspace(x1, x2, ngridx)
yi = np.linspace(y1, y2, ngridy)

points = np.array(list(product(xi, yi)))
x_grid = decode(points)
x_grid.shape

In [ ]:

# UE
masks = build_masks(DEFAULT_MASKS)
for name, mask in masks.items():
    estimator = build_estimator('mcdue_masked', model, dropout_mask=mask)
    estimations = estimator.estimate(x_grid)
    zi = estimations.reshape((ngridx, ngridy))

    fig, ax1 = plt.subplots(figsize=(16, 12))
    ax1.contour(xi, yi, zi, levels=14, linewidths=0.5, colors='k')
    cntr1 = ax1.contourf(xi, yi, zi, levels=14, cmap="gray")
    fig.colorbar(cntr1, ax=ax1)

    ax1.scatter(x, y, c=y_train.squeeze(), alpha=0.5)
    ax1.set(xlim=(x1, x2), ylim=(y1, y2))
    ax1.set_title('%s grid and contour (%d points, %d grid points)' %
                  (name, len(x), ngridx * ngridy))

In [ ]:
decode(np.array([[10, 0.7]]))
